# Modelo de Ising-Kawasaki

En este documento se presenta un desglose completo del código de un modelo de Ising basado en la dinámica de Kawasaki. El código se ha dividido en celdas principalmente para facilitar la explicación del contenido de las mismas, ya que en realidad, para obtener resultados del programa la celda que tiene que ejecutarse el la última de todas (y todas las anteriores). Ésto se ha hecho con el propósito de facilitar la manipulación del mismo, sin tener q desplazarse a celdas anteriores para modificar parámetros, o graficar observables de ningún tipo.

## Introducción

El enfoque para este modelo es muy similar al del modelo clásico de Ising (dinámica de Glauber), donde simplemente aplicamos la dinámica de kawasaki. La principal diferencia en términos generales, es que como vamos a ver en este caso, al intercambiar espines, la magnetización total del sistema va a conservarse en todos los casos. La energía por otro lado, al aplicar el algoritmo de Metrópolis sí que va a disminuir hasta estabilizarse o por el contrario, alcanzará un estado estacionario. El que se dé un comportamiento u otro, estará muy estrechamente relacionado con el valor de la temperatura. Una vez modelado el sistema básico, se han calculadoalgunos observables y magnitudes de interés con el fin de extraer información valiosa del modelo. 

## Procedimiento computacional

En el enunciado del problema se nos pedían 7 tareas a realizar como mínimo en el modelo. A lo largo de esta sección se desglosará como se han enfrentado dichas tareas y se explicará el procedimiento. Las tareas en cuestión son:

1. Simular para varios valores de la temperatura la dinamica de este modelo. Representar varios fotogramas asociados a varias temperaturas.

2. Obtener la curva de magnetizacion por partícula calculada ‘por dominios’ como funcion de la temperatura para varios tamanos del sistema (e.g. N = 32, 64, 128) promediando sobre un numero suficiente de pasos Monte Carlo para que el error sea razonablemente pequeno. Dicho promedio ‘por dominios’ se realiza –en caso de una magnetizacion inicial nula– calculando la magnetización en cada una de las mitades (superior e inferior) del sistema.

3. Calcular densidad de partículas promedio en la direccion y para varias temperaturas.

4. Calcular la energía media por partícula como funcion de la temperatura para los diferentes tamaños.

5. Calcular el calor específico a partir de las fluctuaciones de la energía,

$$
c_N = \frac{1}{N^2 T^2} \left[ \langle E^2 \rangle - \langle E \rangle^2 \right]
$$

como funcion de la temperatura para los diferentes tamaños y determinar la temperatura crítica. Para estimar
dicha temperatura se ha de obtener, para cada valor del tamaño N, el maximo del calor específico, $\text{T}_{c}\text{(N)}$, y
estudiar su comportamiento con N para extrapolar su valor en el l ́ımite N → ∞.

6. Calcular la susceptibilidad magnética a partir de las fluctuaciones de la magnetización en cada dominio,

$$
\chi_N = \frac{1}{N^2 T} \left[ \langle M^2 \rangle - \langle M \rangle^2 \right]
$$

y determinar la temperatura crítica. Para estimar dicha temperatura se ha de obtener, para cada valor del tamaño N, el maximo de la susceptibilidad magnética, $\text{T}_{c}\text{(N)}$, y estudiar su comportamiento con N para extrapolar su valor en el límite N → ∞.

7. Realizar los puntos 1-6 anteriores partiendo de una magnetizacion no nula. Recordemos que la magnetización por partícula puede tomar valores entre −1 y 1, i.e. m ∈ [−1, 1]. Por tanto si fijamos m = $\text{m}_0$ el promedio ‘por dominios’ de la magnetizacion se realizará, en este caso, promediando por separado la fracción x = (1 + $\text{m}_0$)/2
(con x ∈ [0, 1]) inferior del sistema, y la fraccion 1 − x superior del mismo. De esta manera, para T → 0,
tendremos:

$$
m = x(+1) + (1 − x)(−1) = m_0.
$$

Observamos que si $\text{m}_0$ = 0 tenemos que x = 1/2, es decir, tenemos que promediar la magnetizacion por separado en la mitad inferior y en la mitad superior del sistema, como hab ́ıamos explicado en el punto 2. Representar la curva de magnetizacion frente a temperatura y comprobar que es discontinua por debajo de una temperatura crítica. Dicha discontinuidad se hara más pronunciada conforme el tamaño del sistema sea mayor.

### Celda 1:

En la primera celda, como suele ser habitual se han incluido los imports necesarios para la ejecución del código. Algunos que nos pueden llamar la atención son _time_, _tqdm_, _os_, _h5py_, _subprocess_, _threading_ y _re_. _Time_ y _tqdm_, por un lado, se han utilizado para el cálculo del tiempo de ejecución y la generación de una barra de progreso, respectivamente. Obviamente esto en innecesario pero útil para saber si una ejecución va a tardar 2 minutos o 3 horas. Por otro lado, _os_ se ha utilizado para obtener información del dispositivo y detectar el número de hebras disponibles (aunque como veremos no ha sido de utilidad). Y por último _subprocess_, _threading_ y _re_, se utilizan para la generación del video de la simulación. 

Tras esto definimos un rng seguro, basado en la entropía del sistema para una mayor aleatoriedad. La ventaja de este es que se podría utilizar para paralelizar con numba o PyOMP si fuera posible. 

Finalmente, se define la función _establecer_numero_hilos(threads_percentage)_, que analiza la CPU del usuario, para determinar el número de hebras, y utiliza un porcentaje de los mismo, _threads_percentage_, elegido por el usuario. 

In [24]:
# Celda 1: Imports, semilla y configuración de hilos

import numpy as np                      # Celdas: 2,3,4,5,6,7,8 (NP arrays, random choice with rng)
import matplotlib.pyplot as plt         # Celdas: 2,6,7,8 (Visualización, gráficos, animación)
import time                             # Celda: 5 (Medición de tiempos)
from tqdm import tqdm                   # Celda: 5 (Barra de progreso, opcional)
from numba import njit, set_num_threads, get_num_threads
import os                               # Para obtener el número de hilos disponibles
import h5py                             # Para guardar resultados en formato HDF5
import subprocess, threading, re, math # Última celda para la generación del video

np.set_printoptions(threshold=np.inf, linewidth=200)

# ─── Configuración de semilla para reproducibilidad ──────────────────────────
seed = None                             # None = usar entropía del sistema
rng  = np.random.default_rng(seed)      # PCG64 RNG: seguro y adecuado para simulaciones

def establecer_numero_hilos(threads_percentage):
    #Establecemos el número de hilos a usar asegurándonos de que no exceda el número de hilos disponibles ni sea menor a 1
    n_threads_available = os.cpu_count()
    if n_threads_available is None:
        n_threads_available = 1  # Si no se puede determinar, usar al menos 1 hilo
    threads_percentage = max(1, min(100, threads_percentage))
    set_num_threads(int(n_threads_available*(threads_percentage / 100.0)))
    n_threads = get_num_threads()
    print(f"Usando {n_threads} hilos de {n_threads_available} disponibles ({threads_percentage}%).")


### Celda 2:

Aquí se definen la mayoría de los observables termodinámicos del sistema. En primer lugar tenemos dos pares de funciones muy parecidas; _single_energy(config, J)_ y _new_energy(J, frames:np.ndarray)_, y _single_magnetization(config: np.ndarray)_ y _new_magnetization(frames: np.ndarray)_. Estas calculan la energía y la magnetización, de una sola configuración y de un vector de configuraciones, respectivamente. Las cuatro son muy simples, son una aplicación directa de la definición de estos observables. 

Por otro lado tenemos _domain_magnetization(frames: np.ndarray, density)_, que calcula la magnetización por dominios. Para ello define un límite, que detemina el tamaño de los diferentes dominios (se definen los dominios como las regiones de espines $\pm$ 1 cuando el sistema se estabiliza para T $\rightarrow$ 0). Esta función devuelve un array de magnetizaciones de ambos dominios.

Se definen también dos funciones que no son observables exactamente, _linear_regression_slope(x, y)_ y _acceptance_slope(acceptance: np.ndarray, threshold: float)_. La primera simplemente calcula una regresión lineal y devuelve la pendiente, y la segunda, compara esa pendiente con un umbral _threshold_, si es menor devuelve True, si no False. Esto se utiliza para la condición de parada del programa. Para ello necesita definir otra función, _calculate_acceptance(frames: np.ndarray)_, que calcula la tasa de aceptación en cada frame (tasa de éxito de intercambio de espines en cada barrido completo). Cuando el módulo de la pendiente de la tasa de aceptación se acerque a 0 (cuando el sistema se estabilice), la simulación se detendrá. 

Por otro lado, se calculan también el calor específico y la susceptibilidad magnética, _specific_heat(energy: np.ndarray, temperature: float, L: int)_ y _magnetic_susceptibility(domain_magnetization: np.ndarray, temperature: float, L: int)_. Por último, tenemos la densidad media de partículas en el eje y, _calcular_densidad_party(frames: np.ndarray)_. Esto es básicamente la densidad de espines con valor +1, en función de y.


In [25]:
# Celda 2: Definición de algunas funciones de observables

def single_energy(config, J):
    """
    Calcula la energía total del modelo de Ising 2D con contorno periódico.
    """
    # Enlaces derecha e inferior para contar cada par una sola vez
    right = np.roll(config, -1, axis=1)
    down  = np.roll(config, -1, axis=0)
    energy = -J * np.sum(config * (right + down))
    return energy


def new_energy(J, frames:np.ndarray) -> np.ndarray:
    """
    Calcula la energía total del modelo de Ising 2D con contorno periódico.
    """
    # Enlaces derecha e inferior para contar cada par una sola vez
    nframes, H, W = frames.shape
    energy = np.zeros(nframes, dtype=np.float64)  # Array para almacenar la energía de cada frame
    for frame in range(nframes):
        config = frames[frame, :, :]
        right = np.roll(config, -1, axis=1)
        down  = np.roll(config, -1, axis=0)
        energy[frame] = -J * np.sum(config * (right + down))
    return energy


def single_magnetization(config: np.ndarray) -> float:
    """
    Calcula la magnetización total del sistema.
    """
    H, W = config.shape
    return np.sum(config) / (H * W)  # Magnetización normalizada


@njit
def new_magnetization(frames: np.ndarray) -> np.ndarray:
    """
    Calcula la magnetización total del sistema para cada frame.
    """
    nframes, H, W = frames.shape
    magnetizations = np.zeros(nframes, dtype=np.float64)  # Array para almacenar la magnetización de cada frame
    for frame in range(nframes):
        config = frames[frame, :, :]
        magnetizations[frame] = np.sum(config)/(H*W)
    return magnetizations


def domain_magnetization(frames: np.ndarray, density) -> np.ndarray:
    """
    Calcula la magnetización del sistema para el dominio superior e inferior. Para ello esta función
    las calcula y las devuelve en un array de dos dimensiones, 2xnframes, donde la primera fila
    corresponde a la magnetización del dominio superior y la segunda fila a la del dominio inferior.
    """
    nframes, H, W = frames.shape
    magnetizations = np.zeros((2, nframes), dtype=np.float64)  # Array para almacenar la magnetización de cada frame
    lim = int(np.rint(H*density))
    for frame in range(nframes):
        config = frames[frame, :, :]
        magnetizations[0, frame] = np.sum(config[0:(H - lim), :])/((H - lim)*W)
        magnetizations[1, frame] = np.sum(config[(H - lim):H, :])/(lim*W)
    return magnetizations


def linear_regression_slope(x, y):
    """
    Calcula la pendiente de la recta de regresión lineal ajustada a los puntos (x, y)
    usando la función np.polyfit de NumPy.

    Parámetros
    ----------
    x : array_like, shape (n,)
        Vector de coordenadas independientes.
    y : array_like, shape (n,)
        Vector de coordenadas dependientes.

    Devuelve
    -------
    slope : float
        Pendiente de la recta de regresión lineal.

    Lanza
    -----
    ValueError
        Si x e y no tienen la misma longitud o si n < 2.
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    if x.shape != y.shape:
        raise ValueError("Los vectores x e y deben tener la misma longitud")
    if x.size < 2:
        raise ValueError("Se necesitan al menos dos puntos para ajustar una recta")

    # np.polyfit devuelve [pendiente, intercepto] para grado=1
    slope, _ = np.polyfit(x, y, 1)
    return slope


def acceptance_slope(acceptance: np.ndarray, threshold: float) -> bool:
    """
    Comprueba la pendiente de la tasa de aceptación y si es menor que un umbral toma valor True.
    """
    x = np.arange(acceptance.size)
    slope = abs(linear_regression_slope(x, acceptance))
    if slope < threshold:
        print(f"Pendiente de la tasa de aceptación: {slope} < {threshold:.4f}")
        return True
    else:
        return False
    
    
def specific_heat(energy: np.ndarray, temperature: float, L: int) -> float:
    """
    Calcula el calor específico a partir de la energía y la temperatura.
    """
    frames = len(energy)                                                # Número de frames
    stabilized_energy = energy[int(frames*0.75):]                       # Tomamos el 25% final de los frames para medir solo la parte estable de la energía
    Squared_energy = np.mean(stabilized_energy**2)                      # <E^2>
    Mean_energy= np.mean(stabilized_energy)                             # <E>
    SH = (Squared_energy-Mean_energy**2)/((L**2)*(temperature**2))      # Calor específico
    return SH                                                           # Devolvemos el calor específico como un número flotante, que es la media del calor específico de todos los frames estables.


def magnetic_susceptibility(domain_magnetization: np.ndarray, temperature: float, L: int) -> np.ndarray:
    """
    Calcula la susceptibilidad magnética a partir de la magnetización del dominio superior e inferior.
    """
    frames = domain_magnetization.shape[0]                                          # Número de frames
    stabilized_magnetization = domain_magnetization[:, int(frames*0.75):]           # Tomamos el 25% final de los frames para medir solo la parte estable de la magnetización
    UpperMagn = stabilized_magnetization[0, :]
    LowerMagn = stabilized_magnetization[1, :]
    Mean_Upper_Magn = np.mean(UpperMagn)
    Mean_Lower_Magn = np.mean(LowerMagn)
    Squared_Mean_Upper_Magn = np.mean(UpperMagn**2)
    Squared_Mean_Lower_Magn = np.mean(LowerMagn**2)
    Upper_MS = (Squared_Mean_Upper_Magn-(Mean_Upper_Magn**2))/((L**2)*temperature)
    Lower_MS = (Squared_Mean_Lower_Magn-(Mean_Lower_Magn**2))/((L**2)*temperature)
    Mean_MS = (Upper_MS + Lower_MS)/2

    return Mean_MS


@njit
def calculate_acceptance(frames: np.ndarray) -> np.ndarray:
    
    nframes, H, W = frames.shape
    # True donde el espín cambió respecto al sweep anterior
    changes = frames[1:] != frames[:-1]               # shape (nframes-1, H, W)
    diff_counts = changes.reshape(nframes-1, -1).sum(axis=1)
    # Cada swap válido intercambia dos posiciones
    accepted_swaps = diff_counts / 2
    # Nº de intentos de swap por sweep ≈ H*W
    attempts = H * W
    acceptance = accepted_swaps / attempts
    return acceptance


@njit
def calcular_densidad_party(frames: np.ndarray) -> np.ndarray:
    """
    Calcula la densidad de partículas en cada frame.
    """
    nframes, H, W = frames.shape                # Obviamente H, W = L, L
    config = frames[-1, :, :]                   # Última columna de cada frame
    density= np.zeros(H, dtype=np.float64)      # Array para almacenar la densidad de partículas

    # Sumamos los espines a lo largo del eje x, y lo almacenamos en un array
    for i in range(H):
        s = 0
        for j in range(W):
            s += (config[i, j] + 1)/2           # Convertimos espines -1, +1 a densidad 0, 1
        density[i] = s/W                        # Densidad media de partículas en la fila i
    return density

### Celda 3:

En esta celda se definen tres métodos para generar la configuración inicial de espines en una red cuadrada de tamaño $L \times L$ con una densidad dada de espines $+1$. Cada función garantiza que la magnetización total cumpla con el valor deseado y permite distintos tipos de condiciones de contorno o configuraciones asimétricas:

- _random_config_boundary(L, density)_: 
  1. Crea una matriz config de tamaño $L\times L$ inicializada con todos los espines en $+1$.  
  2. Fija la fila superior a $-1$ para imponer condiciones de contorno fijas en la dirección vertical.  
  3. Inserta espines $-1$ aleatoriamente (excluyendo la fila superior) hasta que la magnetización total normalizada  
     $$
       m = \frac{1}{L^2}\sum_{i,j} s_{i,j}
     $$  
     coincida con el valor deseado $2\,\text{density}-1$.  

- _random_config_non_boundary(L, density)_: 
  1. Crea una matriz config con todos los espines en $+1$.  
  2. Inserta espines $-1$ en posiciones completamente aleatorias (toda la red) hasta alcanzar la magnetización  
     normalizada $2\,\text{density} - 1$, sin ninguna fila fija (condiciones periódicas en ambas direcciones).  

- _asimmetric_config(L, density)_:  
  1. Inicializa la red con todos los espines en $+1$.  
  2. Calcula el número total de espines $-1$ necesario: $(1-\text{density})\times L^2$.  
  3. Recorre la matriz fila por fila, colocando \(-1\) de forma determinista en las primeras posiciones hasta  
     agotar el número calculado, obteniendo así una **configuración asimétrica**.  

- _init_config(destino, L, density, Boundary_conditions, Asimmetric)_:  
  1. Elige una de las tres funciones anteriores en función de los flags Boundary_conditions y Asimmetric.  
  2. Genera la configuración inicial config.  
  3. Dibuja la red en escala de grises con matplotlib y la guarda en el fichero PNG indicado por destino.  
  4. Devuelve la matriz config para usarla en el resto de la simulación.

In [26]:
# Celda 3: Inicialización de configuraciones

def random_config_boundary(L, density):   
    config = np.ones((L, L), dtype=int)
    config[0, :] = -1  # Fila superior
    while single_magnetization(config) != (2*density - 1):  # Aseguramos que la magnetización sea igual a la densidad deseada
        i, j = np.random.randint(1, L-1), np.random.randint(0, L)  # Elegir un espín aleatorio
        config[i, j] = -1
    return config


def random_config_non_boundary(L, density):   
    config = np.ones((L, L), dtype=int)      # Inicializar toda la red con espines +1
    while single_magnetization(config) != (2*density - 1):  # Aseguramos que la magnetización sea igual a la densidad deseada
        i, j = np.random.randint(0, L), np.random.randint(0, L)  # Elegir un espín aleatorio
        config[i, j] = -1
    return config


def asimmetric_config(L, density):
    config = np.ones((L, L), dtype=int)      # Inicializar toda la red con espines 1

    # Ahora simplemente hacemos que el porcentaje "density" superior sean -1s:
    # Para ello la forma más sencilla es:
    #   · Calcular el número de espines -1
    #   · Recorrer la malla llenando de -1

    DownSpins = (1-density)*L*L # Número de espines -1 a colocar

    for i in range(L):
        for j in range(L):
            if DownSpins > 0:
                config[i, j] = -1 
                DownSpins += -1
            else:
                return config


# Ahora creamos una función para guardar la configuración inicial en un archivo .png, y que devuelva la configuración
def init_config(destino, L, density, Boundary_conditions, Asimmetric):
    """
    Guarda la configuración inicial en un archivo .png.
    """
    if not Asimmetric:
        if Boundary_conditions:
            config = random_config_boundary(L, density)  # Generar configuración aleatoria y fijar condiciones de frontera 
        else:
            config = random_config_non_boundary(L, density)
    else:
        config = asimmetric_config(L, density)
    plt.figure(figsize=(5, 5))
    plt.imshow(config, cmap='gray', interpolation='nearest')
    plt.title('Configuración inicial aleatoria')
    plt.axis('off')
    plt.savefig(destino, dpi=300, bbox_inches='tight')
    plt.close()
    return config

### Celda 4:

En esta celda se implementa la dinámica de Kawasaki mediante Numba (@njit) para maximizar el rendimiento. Se incluyen tres funciones principales:

- _delta_E_kawasaki(config, i, j, k, l, J, L)_:
  1. Dada una configuración config y dos posiciones vecinas $(i,j)$ y $(k,l)$, calcula la diferencia de energía $\Delta E$ al intercambiar sus espines.  
  2. Suma las contribuciones de los cuatro vecinos de cada espín (excluyendo el par intercambiado).  
  3. Calcula la energía inicial  
     $$
       E_1 = s_{i,j}\sum_{\langle n\rangle} s_n \;+\; s_{k,l}\sum_{\langle m\rangle} s_m  
     $$
     y la energía tras el intercambio  
     $$
       E_2 = s_{k,l}\sum_{\langle n\rangle} s_n \;+\; s_{i,j}\sum_{\langle m\rangle} s_m  
     $$  
  4. Devuelve  
     $$  
       \Delta E = -J\,(E_2 - E_1)\,.  
     $$

- _sweep_kawasaki_boundary(config, L, J, Beta)_:
  1. Recorre $(L-2)\times L$ intentos de intercambio, excluyendo las filas superior e inferior (bordes fijos).  
  2. Para cada intento, elige un espín $(i,j)$ aleatorio en las filas intermedias.  
  3. Selecciona un vecino $(n_i,n_j)$ mediante offsets aleatorios, cuidando no salirse de los bordes fijos.  
  4. Si los dos espines difieren, calcula $\Delta E$ y acepta el intercambio si $\Delta E \le 0$ o si  
     $$  
       \exp(-\Delta E\,\Beta) > \mathrm{rand}()  
     $$

- _sweep_kawasaki_non_boundary(config, L, J, Beta)_  
  1. Similar al anterior, pero recorre $L\times L$ intentos y permite condiciones completamente periódicas (toda la red).  
  2. No hay distinción de bordes; los offsets se eligen siempre entre los cuatro vecinos.  
  3. Aplica la misma regla de aceptación de Metrópolis usando $\Delta E$ y $\Beta = 1/T$.  

Este bloque optimizado en Numba acelera notablemente los barridos Monte Carlo al compilar estas funciones en código nativo.  


In [27]:
# Celda 4: Dinámica de Kawasaki y funciones numba

@njit
def delta_E_kawasaki(config, i, j, k, l, J, L):
    """
    Calcula el cambio de energía ΔE para un intercambio de espines en la dinámica de Kawasaki.
    """
    # Calculamos los vecinos de (i, j) y (k, l) excluyendo el espín que se va a intercambiar
    neighbors_ij = config[i,(j-1)%L] + config[(i-1)%L,j] + config[(i+1)%L,j] + config[i,(j+1)%L] - config[k, l]
    neighbors_kl = config[k,(l-1)%L] + config[(k-1)%L,l] + config[(k+1)%L,l] + config[k,(l+1)%L] - config[i, j]

    #Calculamos la energía de la configuración inicial
    E_1 = config[i,j]*neighbors_ij + config[k,l]*neighbors_kl

    #Calculamos la energía de la configuración final
    E_2 = config[k,l]*neighbors_ij + config[i,j]*neighbors_kl

    #Calculamos el cambio de energía
    delta_E = -J*(E_2 - E_1)

    return delta_E


#Paso de la simulación

@njit
def sweep_kawasaki_boundary(config, L, J, Beta):
    for k in range(((L-2)*L)):

        #Seleccionamos un espín aleatorio (i, j) de la red excluyendo las filas superior e inferior
        i, j = np.random.randint(1, L-1), np.random.randint(0, L)

        # Escribimos el espin seleccionado en un archivo para depuración
        # Definimos los offsets para los vecinos (arriba, abajo, izquierda, derecha)
        offsets = np.array([(1, 0), (0, 1), (0, -1),  (-1, 0)], dtype=np.int64)

        # Ahora seleccionamos un offset aleatorio que decidirá si escogemos un vecino arriba, abajo, izquierda o derecha
        #Hay que mantener la condición de los espines superior e inferior.
        # Entonces lo que hacemos es limitar los offsets a 3 si estamos en la fila superior o inferior, y a 4 si estamos en el resto de la red.
        # Y luego forzamos que si está en la fila
        if i == 1:
            di, dj = offsets[np.random.randint(0, 3)]
        elif i == L-2:
            di, dj = offsets[np.random.randint(1, 4)]
        else:
            di, dj = offsets[np.random.randint(0, 4)]

        # Ahora podemos calcular la posición exacta del espín vecino
        ni, nj = (i + di) % L, (j + dj) % L

        # Escribimos el espín vecino en el archivo para depuración
        # Ahora que tenemos la posición del espín vecino, comprobamos que no sea el mismo espín (i, j) que el vecino (ni, nj)
        if config[i, j] != config[ni, nj]:
            delta_E = delta_E_kawasaki(config, i, j, ni, nj, J, L)

            # Ahora que tenemos el ΔE, podemos decidir si aceptamos o no el movimiento
            # La condición básicamente es que para ΔE <= 0, aceptamos el movimiento, ya que de ser así la probabilidad de aceptación es 1.
            # Si ΔE > 0, aceptamos el movimiento con probabilidad p = exp(-ΔE/T), y lo más eficiente es generar un número aleatorio entre 0 y 1 y comparar con p,
            # ya que si el número aleatorio es menor o igual que p, aceptamos el movimiento.
            if delta_E <= 0 or np.random.rand() < np.exp(-delta_E * Beta):

                # Intercambiar espines
                config[i, j], config[ni, nj] = config[ni, nj], config[i, j]

@njit
def sweep_kawasaki_non_boundary(config, L, J, Beta):            
    for k in range(L*L):

        #Seleccionamos un espín aleatorio (i, j) de la red excluyendo las filas superior e inferior
        i, j = np.random.randint(0, L), np.random.randint(0, L)

        # Escribimos el espin seleccionado en un archivo para depuración
        # Definimos los offsets para los vecinos (arriba, abajo, izquierda, derecha)
        offsets = np.array([(1, 0), (0, 1), (0, -1),  (-1, 0)], dtype=np.int64)

        # Ahora seleccionamos un offset aleatorio que decidirá si escogemos un vecino arriba, abajo, izquierda o derecha
        #Hay que mantener la condición de los espines superior e inferior.
        # Entonces lo que hacemos es limitar los offsets a 3 si estamos en la fila superior o inferior, y a 4 si estamos en el resto de la red.
        # Y luego forzamos que si está en la fila
        di, dj = offsets[np.random.randint(0, 4)]

        # Ahora podemos calcular la posición exacta del espín vecino
        ni, nj = (i + di) % L, (j + dj) % L

        # Escribimos el espín vecino en el archivo para depuración
        # Ahora que tenemos la posición del espín vecino, comprobamos que no sea el mismo espín (i, j) que el vecino (ni, nj)
        if config[i, j] != config[ni, nj]:
            delta_E = delta_E_kawasaki(config, i, j, ni, nj, J, L)

            # Ahora que tenemos el ΔE, podemos decidir si aceptamos o no el movimiento
            # La condición básicamente es que para ΔE <= 0, aceptamos el movimiento, ya que de ser así la probabilidad de aceptación es 1.
            # Si ΔE > 0, aceptamos el movimiento con probabilidad p = exp(-ΔE/T), y lo más eficiente es generar un número aleatorio entre 0 y 1 y comparar con p,
            # ya que si el número aleatorio es menor o igual que p, aceptamos el movimiento.
            if delta_E <= 0 or np.random.rand() < np.exp(-delta_E * Beta):

                # Intercambiar espines
                config[i, j], config[ni, nj] = config[ni, nj], config[i, j]


### Celda 5:

Esta celda contiene la función _plot_observables(destino, J, density)_, que:

1. Carga los datos:
   - Abre el archivo HDF5 configs.h5 en la carpeta destino y extrae el array frames de dimensiones (nframes, L, L).  
   - Lee el atributo thin para conocer la frecuencia de guardado.

2. _Tasa de aceptación:
   - Calcula la tasa de aceptación en cada sweep usando calculate_acceptance(frames).  
   - Crea el array sweeps multiplicando cada índice por thin.  
   - Genera y guarda la gráfica _acceptance_rate.png_ de aceptación vs. sweep.

3. Energía:  
   - Calcula la energía de cada frame con new_energy(J, frames).  
   - Ajusta el array de sweeps y dibuja la gráfica _energy.png_ de energía vs. sweep.

4. Magnetización global:  
   - Obtiene la magnetización por frame con new_magnetization(frames).  
   - Dibuja la gráfica _magnetization.png_ de magnetización vs. sweep.

5. Magnetización por dominios:  
   - Calcula la magnetización del dominio superior e inferior usando _domain_magnetization(frames, density)_.  
   - Dibuja _domain_magnetization.png_, con curvas separadas para cada dominio.

6. Densidad de partículas en la dirección $y$:  
   - Extrae la densidad con _calcular_densidad_party(frames)_.  
   - Ajusta el eje vertical _y_array_ multiplicando por _thin_.  
   - Dibuja _DensityAlongYAxis.png_ de densidad vs. posición _y_.

7. Energía media por partícula:  
   - Calcula el valor final de energía media por partícula  
     $$
       \frac{E_{\text{ultimo}}}{\,L^2 \cdot \tfrac{1}{2}(m_{\text{ultimo}}+1)}  
     $$  
   - Devuelve un diccionario con los observables clave:  
     - _density_  
     - _mean_energy_per_particle_  
     - _energy_  
     - _domain_magnetization_  

Cada bloque de ploteo se encarga de cerrar la figura (_plt.close()_) para liberar memoria y evitar solapamiento de gráficos.  


In [28]:
# Celda 5: Definición función de ploteo de observables

def plot_observables(destino, J, density):

    # Primero de todo, vamos a cargar los datos de las configuraciones guardadas en el archivo HDF5
    with h5py.File(os.path.join(destino, 'configs.h5'), 'r') as f:
        frames = f['configs'][:]                    # np.ndarray (nframes, H, W)
        thin = f.attrs['thin']                      # Frecuencia de guardado
    nframes, H, W = frames.shape                    # nframes = saved_sweeps

    # ─── Acceptance rate ────────────────────────────────

    acceptance = calculate_acceptance(frames)
    sweeps = np.arange(len(acceptance))             # Array de sweeps
    for i in range(len(sweeps)):
        sweeps[i] = sweeps[i]*thin

    plt.figure(figsize=(6, 4))
    plt.plot(sweeps, acceptance, linestyle='-')
    plt.xlabel('Sweep')
    plt.ylabel('Acceptance rate')
    plt.title('Evolución de la tasa de aceptación (Kawasaki)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'acceptance_rate.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # ─── Energía ────────────────────────────────

    energies = new_energy(J, frames)                # Calcular energía de cada frame
    n_sweeps_array = np.arange(len(energies))       # Array de sweeps
    for i in range(nframes):
        n_sweeps_array[i] = n_sweeps_array[i]*thin

    plt.figure(figsize=(6, 4))
    plt.plot(n_sweeps_array, energies, linestyle='-')
    plt.xlabel('Sweep')
    plt.ylabel('Energía')
    plt.title('Energía del sistema (Kawasaki)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'energy.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # ─── Magnetización ────────────────────────────────

    magnetizations = new_magnetization(frames)                      # Calcular magnetización de cada frame
    n_sweeps_array = np.arange(len(magnetizations))                 # Array de sweeps
    for i in range(nframes):
        n_sweeps_array[i] = n_sweeps_array[i]*thin

    plt.figure(figsize=(6, 4))
    plt.plot(n_sweeps_array, magnetizations, linestyle='-')
    plt.xlabel('Sweep')
    plt.ylabel('Magnetización')
    plt.title('Magnetización del sistema (Kawasaki)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'magnetization.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # ─── Magnetización del dominio superior e inferior ────────────────────────────────

    domain_magnetizations = domain_magnetization(frames, density)   # Calcular magnetización del dominio superior e inferior
    n_sweeps_array = np.arange(len(domain_magnetizations[0, :]))    # Array de sweeps

    for i in range(len(n_sweeps_array)):
        n_sweeps_array[i] = n_sweeps_array[i]*thin
    
    plt.figure(figsize=(6, 4))
    plt.plot(n_sweeps_array, domain_magnetizations[0, :], linestyle='-', label='Dominio superior')
    plt.plot(n_sweeps_array, domain_magnetizations[1, :], linestyle='-', label='Dominio inferior')
    plt.xlabel('Sweep')
    plt.ylabel('Magnetización')
    plt.title('Magnetización del dominio superior e inferior (Kawasaki)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'domain_magnetization.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # ─── Densidad de part en dir y ────────────────────

    density = calcular_densidad_party(frames)
    y_array = np.arange(len(density))                           # Eje y de la matriz de configuración
    for i in range(len(y_array)):
        y_array[i] = y_array[i]*thin

    plt.figure(figsize=(6, 4))
    plt.plot(density, y_array, linestyle='-')
    plt.xlabel('Mean Particle Density')
    plt.ylabel('y axis')
    plt.title('Densidad de media partículas en la direccion y')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'DensityAlongYAxis.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # Creamos el array de energía media por partícula 
    mean_energy_per_particle = energies[-1] / (H*W*0.5*(magnetizations[-1] + 1))  # Energía media por "partícula"

    return {'density': density, 'mean_energy_per_particle': mean_energy_per_particle, 'energy': energies, 'domain_magnetization': domain_magnetizations}                                        

# Devolvemos el array de observables (si hay más de uno, puedo usar return {'density': density, 'energy': energies, 'magnetization': magnetizations} para devolver un diccionario con todos los observables)

### Celda 6:
En esta celda se define la función _run_monte_carlo(...)_, que ejecuta el barrido Monte Carlo completo y almacena las configuraciones en un archivo HDF5, lo que nos permite almacenar mucha información de forma muy eficiente. Sus pasos son:

1. Inicialización:
   - Llama a _init_config(...)_ para generar y guardar la configuración inicial en PNG.  
   - Calcula el número de sweeps guardados:  
     $$
       \text{saved\_sweeps} = \lfloor \text{n}\_\text{sweeps} / \text{thin} \rfloor + 1
     $$  
   - Define $ \Beta = 1/T $.  

2. Creación del archivo HDF5:
   - Abre (o crea) _configs.h5_ en modo escritura.  
   - Crea el dataset _configs_ con forma _(saved_sweeps, L, L)_ y tipo _i1_ (espines ±1).  
   - Aplica compresión gzip (_compression_opts=4_) y chunks _(1, L, L)_.  
   - Guarda metadatos: _J_, _T_, _L_, _saved_sweeps_, _thin_.  
   - Escribe la configuración inicial en _dataset[0]_.  

3. Primer sweep:
   - Aplica un sweep de Kawasaki (con o sin fronteras, según _Boundary_conditions_) antes del bucle principal.  

4. Bucle principal de sweeps:
   - Itera _sweep_ de 1 a _n_sweeps_, mostrando barra de progreso _tqdm_.  
   - Cada sweep:  
     - Ejecuta un sweep de Kawasaki.  
     - Si _sweep % thin == 0_, guarda _config_ en _dataset[i]_.  
     - Si ya se ha guardado al menos el 10% de _saved_sweeps_ *y* no es asimétrica,  
       se comprueba la pendiente de la tasa de aceptación en una ventana de datos:  
       - Calcula la tasa de aceptación en ventanas crecientes.  
       - Si la pendiente es menor que _threshold_ o la tasa crece >20%, detiene la simulación anticipadamente.  

5. Ajuste y cierre:
   - Redimensiona el dataset a _(i+1, L, L)_ para ajustar el número real de sweeps guardados.  
   - Actualiza el atributo _saved_sweeps_.  
   - Mide y muestra el tiempo total de simulación.  

6. Post-procesado:
   - Llama a _plot_observables(destino, J, density)_ para graficar los observables y devuelve sus resultados.  


In [29]:
# Celda 6: Bucle Monte Carlo y recolección de datos con HDF5

def run_monte_carlo(L, J, T, n_sweeps, thin, destino, Boundary_conditions, density, max_window, threshold, Asimmetric):

    # ─── Inicialización de la simulación ────────────────────────────────
    
    config = init_config(os.path.join(destino, "init_config.png"), L, density, Boundary_conditions, Asimmetric)  # Guardar configuración inicial
    saved_sweeps = n_sweeps // thin + 1 # Número de sweeps guardados
    # Calcular Beta
    Beta = 1.0 / T

    # Parámetros de guardado

    with h5py.File(os.path.join(destino, 'configs.h5'), 'w') as f:
        # Dataset para las configuraciones: snapshots × L × L, dtype int8
        dataset = f.create_dataset(
            'configs',                      # 1. Nombre del dataset dentro del archivo HDF5
            shape=(saved_sweeps, L, L),     # 2. Dimensiones: n_saved muestras de matrices L×L     
            maxshape=(None, L, L),          # 3. Dimensión máxima: puede crecer indefinidamente en el eje de muestras
            dtype='i1',                     # 4. Tipo de dato: int1 (espines ±1)
            compression='gzip',             # 5. Compresión: algoritmo gzip
            compression_opts=4,             # 6. Nivel de compresión (1=rápido/menos compacto … 9=lento/máximo)
            chunks=(1, L, L),               # 7. Fragmentación (“chunking”): cada bloque es una matriz L×L
        )
        # Metadatos
        f.attrs['J'] = J
        f.attrs['T'] = T
        f.attrs['L'] = L
        f.attrs['saved_sweeps'] = saved_sweeps
        f.attrs['thin'] = thin

        # Guardar configuración inicial ds[0]
        dataset[0, :, :] = config
        if Boundary_conditions:
            sweep_kawasaki_boundary(config, L, J, Beta)
        else:
            sweep_kawasaki_non_boundary(config, L, J, Beta)

        # Barrido Monte Carlo
        i=0
        start_time = time.time()
        for sweep in tqdm(range(1, n_sweeps + 1), desc='MC Sweeps'):  # Esto es una simple barra de progreso, nada más
            # Ahora podemos barrer la red para elegir el par de espines a intercambiar.
            if Boundary_conditions:
                sweep_kawasaki_boundary(config, L, J, Beta)
            else:
                sweep_kawasaki_non_boundary(config, L, J, Beta)

            # Almacenar las configuraciones 
            
            if sweep % thin == 0:  # Guardar cada thin sweeps
                i = sweep // thin
                dataset[i, :, :] = config
                # Ahora creamos la condición de parada, que será cuando la pendiente de la tasa de aceptación sea menor que un umbral (se establice)
                if (i >= int(0.1 * saved_sweeps)) and not Asimmetric:  # Si hemos guardado al menos el 5% de los sweeps y la config es aleatoria
                    # Calcular la tasa de aceptación y detener si es menor que el umbral
                    window = min(max_window, i//4)  # Ventana de datos para la tasa de aceptación (entre el 25% de los sweeps guardados y el máximo establecido)
                    if window >= 3:   
                        dataset_window = dataset[:2, :, :]
                        acceptance_array = calculate_acceptance(dataset_window)
                        initial_acceptance = acceptance_array[-1]  # Último valor de aceptación (debería ser el 0)
                        dataset_window = dataset[i-2:i, :, :]
                        acceptance_array = calculate_acceptance(dataset_window)
                        current_acceptance = acceptance_array[-1]  # Último valor de aceptación (debería ser el 0)
                        dataset_window = dataset[i-window:i, :, :]
                        acceptance_array = calculate_acceptance(dataset_window)
                        if acceptance_slope(acceptance_array, threshold):
                            print(f"Simulación detenida en sweep {sweep} por tasa de aceptación.")
                            break
                        # Esto lo añado si quiero que la simulación se detenga si no va a converger
                        elif current_acceptance >= initial_acceptance*1.2:  # Si la tasa de aceptación ha aumentado un 20% respecto a la inicial
                            print(f"Simulación detenida en sweep {sweep} por tasa de aceptación creciente.")
                            break
            if not Asimmetric:    
                if sweep == n_sweeps:  # Si hemos llegado al último sweep, escribimos el último valor de pendiente de aceptación
                    x = np.arange(acceptance_array.size)
                    print("\n")
                    print(abs(linear_regression_slope(x, acceptance_array)))
            

        dataset.resize((i + 1, L, L))  # Ajustar tamaño final del dataset
        f.attrs['saved_sweeps'] = i + 1

        end_time = time.time()

        print(f"Simulación completada en {end_time - start_time:.2f} s")

    # Graficar, guardar y devolver los observables
    return plot_observables(destino, J, density)


### Celda 7:

Esta celda fue generada con ChatGPT por completo, por lo que no puedo desarrollar en profundidad lo que hace cada sección. Sin embargo, intentado comprende algo de lo que hace puedo dar una descripción general de proceso:

- En primer lugar se extran los datos necesarios del archivo _.h5_. 
- Una vez hecho eso se calculan los parámetros del vídeo: fps, resolución, ...
- Luego se intenta detectar el NVENC que es un codificador de NVIDIA que nos va a permitir acelerar por GPU el video, aunque como ya veremos no nos va a ser de demsasiada ayuda.
- Por último, en función de si ha encontrado el NVENC o no, establece unas opciones de vídeo y envía los frames y codifica el vídeo, para después descargarlo en la carpeta donde se le indique. 

In [30]:
# Celda 7: Pipeline GPU-bound con NVENC a partir de HDF5

def generate_video_from_hdf5(HDF5_FILE, DATASET, FILE_OUT, GPU_ID, INTERVAL, TARGET_size, MIN_SIDE, destino):

    # 1) Cargar datos -------------------------------------------------------------------------
    with h5py.File(os.path.join(destino, HDF5_FILE), 'r') as f:
        frames = f[DATASET][::]
    nframes, h0, w0 = frames.shape
    fps = 1000.0 / INTERVAL
    print(f"🎞️  Generando vídeo: {nframes} frames @ {fps:.1f} fps")

    # 2) Calcular resolución de salida --------------------------------------------------------
    w_out, h_out = w0, h0
    if TARGET_size:
        w_out = h_out = TARGET_size

    # Asegurar mínimo NVENC
    if min(w_out, h_out) < MIN_SIDE:
        factor = math.ceil(MIN_SIDE / min(w_out, h_out))
        w_out *= factor
        h_out *= factor

    # Redondear a par
    w_out = (w_out // 2) * 2
    h_out = (h_out // 2) * 2
    if (w_out, h_out) != (w0, h0):
        print(f"↕️  Escalado de resolución: {w0}×{h0} → {w_out}×{h_out}")
    vf_filter = ["-vf", f"scale={w_out}:{h_out}:flags=neighbor"] if (w_out, h_out) != (w0, h0) else []

    # 3) Detectar NVENC -----------------------------------------------------------------------
    encoders = subprocess.run(
        ["ffmpeg", "-hide_banner", "-encoders"],
        capture_output=True, text=True
    ).stdout
    if "h264_nvenc" in encoders:
        print("✅ NVENC (GPU) activado")
        video_opts = [
            "-c:v", "h264_nvenc", "-gpu", str(GPU_ID),
            "-preset", "p1", "-profile:v", "high444p", "-pix_fmt", "yuv444p"
        ]
    else:
        print("⚠️ Usando codificación por CPU (libx264)")
        video_opts = ["-c:v", "libx264", "-preset", "veryslow", "-crf", "0", "-pix_fmt", "yuv420p"]

    # 4) Comando FFmpeg -----------------------------------------------------------------------
    cmd = (
        ["ffmpeg", "-y",
        "-f", "rawvideo", "-pix_fmt", "rgb24",
        "-s", f"{w0}x{h0}", "-r", str(fps), "-i", "-",
        "-progress", "pipe:1", "-loglevel", "error"]
        + vf_filter + video_opts + [f"{os.path.join(destino, FILE_OUT)}.mp4"]
    )

    # 5) Lanzar FFmpeg ------------------------------------------------------------------------
    proc = subprocess.Popen(
        cmd,
        stdin = subprocess.PIPE,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        bufsize = 0
    )

    # 6) Barra de codificación ---------------------------------------------------------------
    pbar_enc = tqdm(total=nframes, desc="🛠️ Codificando", unit="frame")
    def _watch():
        re_time = re.compile(rb"out_time_ms=(\d+)")
        re_fr   = re.compile(rb"frame=(\d+)")
        while True:
            line = proc.stdout.readline()
            if not line:
                break
            m = re_fr.search(line) or re_time.search(line)
            if m:
                val = int(m.group(1))
                done = val if b"frame" in line else min(nframes, int(round(val * fps / 1000)))
                pbar_enc.n = done
                pbar_enc.refresh()

    threading.Thread(target=_watch, daemon=True).start()

    # 7) Enviar frames ------------------------------------------------------------------------
    with tqdm(total=nframes, desc="📤 Enviando frames", unit="frame") as pbar_in:
        for frame in frames:
            # Convertir de Ising (-1,+1) a [0,255] y a RGB
            rgb = np.repeat(((frame + 1) * 127.5).astype(np.uint8)[..., None], 3, axis=2)
            proc.stdin.write(rgb.tobytes())
            pbar_in.update(1)

    proc.stdin.close()
    proc.wait()
    pbar_enc.n = nframes
    pbar_enc.refresh()
    pbar_enc.close()

    print(f"🎉 Vídeo generado: {os.path.join(destino, FILE_OUT)}.mp4 ({w_out}×{h_out})")

### Celda 8:

La función _run_whole_simulation(...)_ orquesta toda la simulación para un conjunto de parámetros dados y automatiza:

1. Configuración de paralelismo:
   - Llama a _establecer_numero_hilos(threads_percentage)_ para ajustar el número de hilos de Numba según el porcentaje solicitado.

2. Creación de carpeta de resultados única:  
   - Asegura que exista la carpeta base _carpeta_.  
   - Construye _destino_base = carpeta/L{L}_J{J}_T{T:.2f}_sweeps{n_sweeps}_threads{threads_percentage}_.  
   - Si ya existe, añade sufijos __({counter})_ hasta encontrar un nombre libre.  
   - Crea finalmente la carpeta _destino_.

3. Ejecución de la simulación Monte Carlo: 
   - Llama a  
     ```
     observables = run_monte_carlo(
       L, J, T, n_sweeps, thin, destino,
       Boundary_conditions, density,
       max_window, threshold, Asimmetric
     )
     ```  
     que devuelve un diccionario con los observables calculados.

4. Generación de vídeo:  
   - Llama a  
     ```
     generate_video_from_hdf5(
       HDF5_FILE, DATASET, FILE_OUT, GPU_ID,
       INTERVAL, TARGET_size, MIN_SIDE, destino
     )
     ```  
     para convertir las configuraciones HDF5 en un archivo MP4 usando NVENC o libx264.

5. Retorno de resultados: 
   - Devuelve el diccionario _observables_ con los arrays de densidad, energía media, magnetización por dominios, etc., listo para posteriores análisis o graficado.  


In [32]:
# Celda 8: Función de simulación completa (wrapper)

def run_whole_simulation(L, J, T, n_sweeps, threads_percentage, thin, Boundary_conditions, density, carpeta, max_window, threshold, Asimmetric, HDF5_FILE, DATASET, FILE_OUT, GPU_ID, INTERVAL, TARGET_size, MIN_SIDE):

    # ─── Establecer el número de hilos a usar ──────────────────────────────────────────────────────────────────────────

    establecer_numero_hilos(threads_percentage)  

    # ─── Definición nombre carpeta ─────────────────────────────────────────────────────────────────────────────────────

                                                                                       # La idea en esta parte es simple, queremos crear
    if not os.path.exists(carpeta):                                                                         # una carpeta de resultados siempre que hagamos una
        os.makedirs(carpeta)                                                                                # simulación, la hayamos hecho antes o no, y que sea única.
    destino_base = os.path.join(carpeta, f"L{L}_J{J}_T{T:.2f}_sweeps{n_sweeps}_threads{threads_percentage}")    # Con este fin, hemos hecho un pequeño bucle que comprueba
    destino = destino_base                                                                                  # si la carpeta ya existe, y si es así, le añade un número 
    counter = 1                                                                                             # al final de la carpeta, para que sea única.
    while os.path.exists(destino):
        destino = f"{destino_base}_({counter})" 
        counter += 1
    os.makedirs(destino)  # Crear una carpeta de destino única

    # Ahora ejecutamos el programa completo

    observables = run_monte_carlo(L, J, T, n_sweeps, thin, destino, Boundary_conditions, density, max_window, threshold, Asimmetric)

    generate_video_from_hdf5(HDF5_FILE, DATASET, FILE_OUT, GPU_ID, INTERVAL, TARGET_size, MIN_SIDE, destino)

    return observables  # Devolver los observables calculados

### Celda 9:

En esta celda se definen varias funciones que generan y guardan gráficas de los distintos observables calculados en la simulación:

- _plot_density_y(dict_densities, T_init, T_step, L_init, L_step, destino)_:
  1. Crea la carpeta _Densities_ dentro de _destino_ si no existe.  
  2. Para cada tamaño de red $L = L_{\text{init}} + j\,L_{\text{step}}$:  
     - Extrae el array _density_y_ de forma $(n_{\text{Temp}}, L)$ del diccionario.  
     - Calcula el eje vertical $y = 0, 1, \dots, L-1$ y los valores de temperatura $T_i = T_{\text{init}} + i\,T_{\text{step}}\quad (i=0,\dots,n_{\text{Temp}}-1).$  
     - Dibuja densidad vs. posición \(y\) para cada temperatura y guarda _DensityAlongYAxis_Temperature_L{L}.png_.  

- _plot_mean_energy_per_particle(mean_energies_per_particle, T_init, T_step, L_init, L_step, destino)_:  
  1. Para cada tamaño de red $L_j = L_{\text{init}} + j\,L_{\text{step}}$:  
     - Extrae la columna correspondiente de _mean_energies_per_particle_ (forma $(n_{\text{Temp}}, n_Ls)$).  
     - Calcula los valores de temperatura $T_i$ como arriba.  
     - Dibuja energía media por partícula vs. $T$ y guarda _MeanEnergyPerParticle_Temperature.png_.  

- _plot_specific_heat(SH, T_init, T_step, L_init, L_step, destino)_:
  1. Para cada tamaño de red $L_j$:  
     - Extrae la columna de _SH_ (capacidad calorífica).  
     - Dibuja $C_N(T)$ vs. $T$ y guarda _SpecificHeat_Temperature.png_.  

- _plot_susceptibility(MS, T_init, T_step, L_init, L_step, destino)_:
  1. Para cada $L_j$:  
     - Extrae la columna de _MS_ (susceptibilidad magnética).  
     - Dibuja $\chi_N(T)$ vs. $T$ y guarda _Susceptibility_Temperature.png_.  

- _plot_Tcrit_L(T_crit_L, L_init, L_step, destino, filename)_
  1. Calcula valores de $L_j$ y recibe el vector _T_crit_L_ con los picos de $C_N$ o $\chi_N$.  
  2. Dibuja temperatura crítica $T_c(L)$ vs. $L$ y guarda el archivo _filename_.  

- _plot_magnetizations_vs_temp(magnetizations_vs_temp, T_init, T_step, L_init, L_step, destino)_:  
  1. Crea la carpeta _Magnetizations_vs_temp_ en _destino_ si no existe.  
  2. Para cada $L_j$:  
     - Extrae los arrays de magnetización del dominio superior e inferior (dimensión $(n_{\text{Temp}},2)$).  
     - Dibuja ambas curvas vs. $T_i$ y guarda _L_{L_j}.png_.  


In [33]:
# Celda 9: Funciones para graficar observables

def plot_density_y(dict_densities, T_init, T_step, L_init, L_step, destino):

    if not os.path.exists(os.path.join(destino, "Densities")):
        carpeta_densities = os.path.join(destino, "Densities")
        os.makedirs(carpeta_densities)

    n_Ls = len(dict_densities)  # Número de tamaños de red

    for j in range(n_Ls):
        density_y = dict_densities[f'Density_L{L_init + j * L_step}']   # np.ndarray (n_Temp, L)
        n_Temp, L = density_y.shape                                     # n_Temp = número de temperaturas, L = tamaño de la red
        y_axis = np.arange(L)                                           # Eje y de la matriz de configuración
        T_values = T_init + np.arange(n_Temp)*T_step 
        plt.figure(figsize=(10, 6))
        for i in range(n_Temp):
            plt.plot(density_y[i,:], y_axis, linestyle='-', label=f'T={T_values[i]:.2f}')
        plt.xlabel('Densidad de partículas')
        plt.ylabel('y axis')
        plt.title(f'Densidad de partículas a lo largo de la dirección y para {n_Temp} temperaturas y L={L}')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(carpeta_densities, f'DensityAlongYAxis_Temperature_L{L}.png'), dpi=300, bbox_inches='tight')
        plt.close()

def plot_mean_energy_per_particle(mean_energies_per_particle, T_init, T_step, L_init, L_step, destino):

    n_Temp, n_Ls = mean_energies_per_particle.shape
    T_values = T_init + np.arange(n_Temp)*T_step
    L_values = L_init + np.arange(n_Ls)*L_step

    plt.figure(figsize=(10, 6))
    for i in range(n_Ls):
        plt.plot(T_values, mean_energies_per_particle[:, i], label= f'L={L_values[i]}', linestyle='-')
    plt.xlabel('Temperatura (T)')
    plt.ylabel('Energía media por partícula')
    plt.title('Energía media por partícula por temperatura a diferentes tamaños de red')
    plt.legend()  
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'MeanEnergyPerParticle_Temperature.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_specific_heat(SH, T_init, T_step, L_init, L_step, destino):

    n_Temp, n_Ls = SH.shape
    T_values = T_init + np.arange(n_Temp)*T_step
    L_values = L_init + np.arange(n_Ls)*L_step    
    
    plt.figure(figsize=(10, 6))
    for i in range(n_Ls):
        plt.plot(T_values, SH[:, i], label= f'L={L_values[i]}', linestyle='-')
    plt.xlabel('Temperatura (T)')
    plt.ylabel('Calor específico')
    plt.title('Calor específico por temperatura a diferentes tamaños de red')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'SpecificHeat_Temperature.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_susceptibility(MS, T_init, T_step, L_init, L_step, destino):
    """
    Plotea la susceptibilidad magnética en función de la temperatura.
    """
    n_Temp, n_Ls = MS.shape
    T_values = T_init + np.arange(n_Temp)*T_step
    L_values = L_init + np.arange(n_Ls)*L_step

    plt.figure(figsize=(10, 6))
    for i in range(n_Ls):
        plt.plot(T_values, MS[:, i], label= f'L={L_values[i]}', linestyle='-')
    plt.xlabel('Temperatura (T)')
    plt.ylabel('Susceptibilidad magnética')
    plt.title('Susceptibilidad magnética por temperatura a diferentes tamaños de red')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, 'Susceptibility_Temperature.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_Tcrit_L(T_crit_L, L_init, L_step, destino, filename):
    """
    Plotea la temperatura crítica en función del tamaño de la red L.
    """
    L_values = L_init + np.arange(T_crit_L.size)*L_step

    plt.figure(figsize=(10, 6))
    plt.plot(L_values, T_crit_L, marker='o', linestyle='-', color='b')
    plt.xlabel('Tamaño de la red (L)')
    plt.ylabel('Temperatura crítica (T_c)')
    plt.title('Temperatura crítica en función del tamaño de la red')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(destino, filename), dpi=300, bbox_inches='tight')
    plt.close()

def plot_magnetizations_vs_temp(magnetizations_vs_temp: np.ndarray, T_init, T_step, L_init, L_step, destino):

    n_Temp, n_Ls, domain = magnetizations_vs_temp.shape
    T_values = T_init + np.arange(n_Temp)*T_step
    L_values = L_init + np.arange(n_Ls)*L_step 

    if not os.path.exists(os.path.join(destino, "Magnetizations_vs_temp")):
        carpeta_magnetizations = os.path.join(destino, "Magnetizations_vs_temp")
        os.makedirs(carpeta_magnetizations)

    for i in range(n_Ls):
        plt.figure(figsize=(10, 6))
        plt.plot(T_values, magnetizations_vs_temp[:, i, 0], linestyle='-', color='b')
        plt.plot(T_values, magnetizations_vs_temp[:, i, 1], linestyle='-', color='r')
        plt.xlabel('Temperatura (T)')
        plt.ylabel('Energía media por partícula')
        plt.title('Energía media por partícula por temperatura a diferentes tamaños de red')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(carpeta_magnetizations, f'L_{L_values[i]}.png'), dpi=300, bbox_inches='tight')
        plt.close()


### Celda 10:

Esta celda define la función _main()_ que coordina múltiples simulaciones variando temperatura y tamaño de red, y genera todos los gráficos finales.

1. Parámetros base del modelo:
   - _L_, _J_, _T_, _n_sweeps_, _density_, _threads_percentage_, _thin_, _Boundary_conditions_, _max_window_, _threshold_, _Asimmetric_.  
   - Determinan la red, dinámica, convergencia y condiciones de frontera.

2. Parámetros para vídeo:
   - _HDF5_FILE_, _DATASET_, _FILE_OUT_, _GPU_ID_, _INTERVAL_, _TARGET_size_, _MIN_SIDE_.

3. Preparación de carpeta de resultados:
   - Crea _results/Simulacion_Multiple(X)_ de forma única.

4. Rangos de temperatura y tamaño:
   - Temperaturas: desde _T_init_ hasta _T_max_ con paso _T_step_.  
   - Tamaños de red: desde _L_init_ hasta _L_max_ con paso _L_step_.  
   - Calcula _n_temps_ y _n_Ls_.

5. Inicialización de los arrays donde van a ir los observables:
   - _mean_energies_per_particle_, _SH_, _MS_, _magnetization_vs_temp_, _dict_densities_.

6. Bucle de simulaciones:  
   - Recorre todos los tamaños de red, y dentro de cada uno de ellos, todas las temperaturas.
   - En cada iteración recoje todos los observables y los almacena. 
   
7. Almacenamiento observables
   - Se genera una carpeta donde se almacenarán todos los observables.
   - Define un último observable _T_crit_L_SH_ y _T_crit_L_MS_, la temperatura crítica con el tamaño de la red.
   - Plotea todos los observables, y los almacena en la carpeta que ha creado.


In [34]:
# Celda 10: Ejecución del programa completo

def main():

    # ─── Parámetros base del modelo ────────────────────────────────────────────────────────────────────────────────────

    L                   = 16                        # (int) Tamaño de la red (LxL)
    J                   = 1.0                       # (float) Constante de interacción (J > 0 para ferromagnetismo)
    T                   = 1.0                       # (float) Temperatura del modelo de Ising 2D
    n_sweeps            = 10000                     # (int) Número de sweeps (barridos) a realizar
    density             = 0.75                       # (float) Densidad de espines +1
    threads_percentage  = 100                       # (int) Porcentaje de hilos a usar (100% = todos los disponibles)
    thin                = 10                        # (int) Frecuencia de guardado de configuraciones (1 = cada sweep, 2 = cada 2 sweeps, etc.)     
    Boundary_conditions = True                      # (bool) Condiciones de frontera (True = eje "y" limitado, False = periódicas)
    max_window          = 1000                      # (int) Ventana de datos para la tasa de aceptación (número de sweeps a considerar para calcular la pendiente de la tasa de aceptación)
    threshold           = 10**-8                    # (float) Umbral de pendiente para aceptar la tasa de aceptación (si la pendiente es menor que este valor, se acepta)
    Asimmetric          = True                      # (bool) Densidad de espines asimetricamente distribuidos
    
    # ─── Parámetros de usuario para la generación del vídeo ────────────────────────────────────────────────────────────

    HDF5_FILE = "configs.h5"                    # Nombre del archivo HDF5 con las configuraciones
    DATASET   = "configs"                       # Nombre del dataset dentro del archivo HDF5
    FILE_OUT  = "simulacion"                    # Nombre del archivo de salida (sin extensión ni ruta)
    GPU_ID    = 0                               # 0 = tu NVIDIA 4050
    INTERVAL  = 50                              # ms entre frames → fps = 1000/INTERVAL
    TARGET_size  = 1440                         # Tamaño del vídeo de salida (1440p, 1080p, etc.)
    MIN_SIDE  = 160                             # mínimo seguro para NVENC (≥ 145 y par)

    # Hemos reducido todo el programa a una sola función que recibe todos los parámetros necesarios.
    # Ahora podemos ejecutar múltiples simulaciones con diferentes parámetros y 
    # vamos a preparar una secuencia de opciones para el usuario, 
    # para que vaya eligiendo qué tipo de simulación quiere hacer.

        

    if not os.path.exists("results"):
        os.makedirs("results")
    destino_base = os.path.join("results", f"Simulacion_Multiple")
    destino = destino_base
    counter = 1
    while os.path.exists(destino):
        destino = f"{destino_base}_({counter})" 
        counter += 1
    os.makedirs(destino)  # Crear una carpeta de destino única

    # Ahora inicializamos los parámetros de temperatura:
    T_init = 2.00                                           # Temperatura inicial
    T_step = 0.125                                          # Paso de temperatura
    T_max  = 2.50                                           # Temperatura máxima

    # Ahora hacemos lo mismo con L:

    L_init = 16                                                                     # Tamaño de red inicial
    L_step = 4                                                                      # Paso de tamaño de red
    L_max  = 20                                                                     # Tamaño de red máximo

    
    n_temps = int(np.rint((T_max - T_init) / T_step)) + 1                                # Número de temperaturas a simular
    n_Ls = int(np.rint((L_max - L_init) / L_step)) + 1                                      # Número de tamaños de red a simular

    # Inicializamos los arrays que vamos a usar para almacenar los resultados de las simulaciones.
    mean_energies_per_particle = np.zeros((n_temps, n_Ls))                          # Inicializar un array para almacenar la energía media por partícula
    SH = np.zeros((n_temps, n_Ls))                                                  # Inicializar un array para almacenar la capacidad calorífica
    MS = np.zeros((n_temps, n_Ls))                                                  # Inicializar un array para almacenar la susceptibilidad magnética (media de los 2 dominios)
    magnetization_vs_temp = np.zeros((n_temps, n_Ls, 2))                            # Inicializar un array para almacenar la magnetización frente a la temperatura
    dict_densities = {}                                                             # Inicializar un diccionario para almacenar las densidades de partículas
    L = L_init                                                                      # Inicializar el tamaño de la red
    j = 0                                                                           # Inicializar el contador de iteraciones para el tamaño de la red


    # Ahora sí, metemos el bucle que ejecutará las simulaciones.
    start_time = time.time()                                                        # Medir el tiempo de ejecución total
    for j in range(n_Ls):

        L = L_init + j * L_step
        density_y = np.zeros((n_temps, L))                                         # Inicializar un array para almacenar la densidad de partículas a lo largo de la dirección y
        for i in range(n_temps):                                                           # Por ejemplo, de 0.5 a 5.0 en incrementos de 0.5

            T = T_init + i * T_step
            # Ahora llamamos a la función general, que nos devuelve los observables en formato array, sin necesidad de acceder a los archivos uno a uno.
            observables = run_whole_simulation(L, J, T, n_sweeps, threads_percentage, thin, Boundary_conditions, density, destino, max_window, threshold, Asimmetric, HDF5_FILE, DATASET, FILE_OUT, GPU_ID, INTERVAL, TARGET_size, MIN_SIDE)
            density_y[i, :] = observables['density']  # Guardar la densidad de partículas a lo largo de la dirección y en el array
            mean_energies_per_particle[i, j] = observables['mean_energy_per_particle']  # Guardar la energía media por partícula
            SH[i, j] = specific_heat(observables['energy'], T, L)  # Calcular el calor específico y guardarlo
            MS[i, j] = magnetic_susceptibility(observables['domain_magnetization'], T, L)  # Calcular la susceptibilidad magnética y guardarla
            magnetization_vs_temp[i, j, :] = observables['domain_magnetization'][:, -1]

        dict_densities[f'Density_L{L}'] = density_y                                 # Guardar la densidad de partículas en el diccionario
    end_time = time.time()                                                          # Medir el tiempo de ejecución total

    # Ahora tenemos un array de densidades de partículas a lo largo de la dirección y, que se ha ido llenando a medida que hemos ido haciendo las simulaciones.

    if not os.path.exists(os.path.join(destino, "Observables")):
        Carpeta_Observables = os.path.join(destino, "Observables")
        os.makedirs(Carpeta_Observables)
    
    plot_density_y(dict_densities, T_init, T_step, L_init, L_step, Carpeta_Observables)                             # Graficar la densidad de partículas a lo largo de la dirección y para cada temperatura                 
    plot_mean_energy_per_particle(mean_energies_per_particle, T_init, T_step, L_init, L_step, Carpeta_Observables)  # Graficar la energía media por partícula para cada temperatura
    plot_specific_heat(SH, T_init, T_step, L_init, L_step, Carpeta_Observables)                                     # Graficar el calor específico para cada temperatura
    plot_susceptibility(MS, T_init, T_step, L_init, L_step, Carpeta_Observables)                                    # Graficar la susceptibilidad magnética para cada temperatura
    plot_magnetizations_vs_temp(magnetization_vs_temp, T_init, T_step, L_init, L_step, Carpeta_Observables)         # Graficar la magnetización frente a la temperatura

    # Ahora hay que dar el valor de la temperatura crítica en función de L.
    # Para esto buscamos la temperatura donde se produce el pico de calor específico.
    T_crit_L_SH = np.zeros(n_Ls)                                                    # Array para almacenar la temperatura crítica para cada tamaño de red
    for j in range(n_Ls):
        T_crit_L_SH[j] = T_init + np.argmax(SH[:, j]) * T_step                      # La temperatura crítica es la temperatura donde el calor específico es máximo
    filenameSH = "Tcrit_L_SH.png"                                                   # Nombre del archivo donde se guardará la gráfica de la temperatura crítica en función del tamaño de la red

    # Ahora ploteamos la temperatura crítica derivada de la capacidad calorífica.
    plot_Tcrit_L(T_crit_L_SH, L_init, L_step, Carpeta_Observables, filenameSH)      # Graficar la temperatura crítica en función del tamaño de la red

    T_crit_L_MS = np.zeros(n_Ls)                                                    # Array para almacenar la temperatura crítica para cada tamaño de red
    for j in range(n_Ls):
        T_crit_L_MS[j] = T_init + np.argmax(MS[:, j]) * T_step                      # La temperatura crítica es la temperatura donde la susceptibilidad magnética es máxima
    filenameMS = 'Tcrit_L_MS.png'

    # Ahora ploteamos la temperatura crítica derivada de la susceptibilidad magnética.
    plot_Tcrit_L(T_crit_L_MS, L_init, L_step, Carpeta_Observables, filenameMS)      # Graficar la temperatura crítica en función del tamaño de la red
    
    print(f"Simulación completada en {end_time - start_time:.2f} s")

main()

Usando 16 hilos de 16 disponibles (100%).


MC Sweeps: 100%|██████████| 10000/10000 [00:00<00:00, 34478.28it/s]


Simulación completada en 0.29 s
🎞️  Generando vídeo: 1001 frames @ 20.0 fps
↕️  Escalado de resolución: 16×16 → 1440×1440
✅ NVENC (GPU) activado


🛠️ Codificando: 100%|██████████| 1001/1001 [00:01<00:00, 531.52frame/s]


🎉 Vídeo generado: results\Simulacion_Multiple_(1)\L16_J1.0_T2.00_sweeps10000_threads100\simulacion.mp4 (1440×1440)
Usando 16 hilos de 16 disponibles (100%).


...

🎉 Vídeo generado: results\Simulacion_Multiple_(1)\L20_J1.0_T2.38_sweeps10000_threads100\simulacion.mp4 (1440×1440)
Usando 16 hilos de 16 disponibles (100%).


MC Sweeps: 100%|██████████| 10000/10000 [00:00<00:00, 18016.62it/s]


Simulación completada en 0.56 s
🎞️  Generando vídeo: 1001 frames @ 20.0 fps
↕️  Escalado de resolución: 20×20 → 1440×1440
✅ NVENC (GPU) activado


🛠️ Codificando: 100%|██████████| 1001/1001 [00:01<00:00, 505.34frame/s]


🎉 Vídeo generado: results\Simulacion_Multiple_(1)\L20_J1.0_T2.50_sweeps10000_threads100\simulacion.mp4 (1440×1440)
Simulación completada en 35.69 s


Como vemos, en esta última celda es donde damos valor a todas las variables, y desde aquí podemos configurar todas las distintas opciones. Ahora que hemos visto cómo funciona el programa a nivel interno, veamos los resultados.


## Resultados y discusión